# DAG full_YA

In [10]:
import re
from typing import Any, Dict, List
import pymysql
# from airflow import DAG
# from airflow.operators.python import PythonOperator
import os 
import datetime
import requests
import time
from sqlalchemy import create_engine
import numpy as np
import openpyxl
import gspread
import gspread_dataframe as gd
from tqdm import tqdm
from sys import exit
import pandas as pd
import sys
import glob
link_to_table = 'https://docs.google.com/spreadsheets/d/1A1TyZPeq9r2HiDi-CHfmTfRpuAQYToib1vqyTx4ZyRE/edit#gid=0'
df = pd.read_excel('/'.join(link_to_table.split('/')[:-1])+'/export?format=xlsx')
PATH, df = [dict(df.loc[x]) for x in list(df.index)], None
i = 0
while len([py for py in glob.glob(f"{PATH[i]['path_downloads']}*.xlsx")]) == 0:
    i +=1
path_downloads = PATH[i]['path_downloads']
path_algoritm = PATH[i]['path_algoritm']
sys.path.append(f'{path_algoritm}Модули')
import constants_for_marketplace_metrics_dags as const
import working_with_marketplace_metrics_dags as wwm

bd_param = const.bd_param
engine_param = const.engine_param
maslow_json = path_downloads + const.maslow_json
gc = gspread.service_account(filename=maslow_json)
args = const.args
os.environ['NO_PROXY'] = 'URL'

def ya_orders_metrics(path_ya_orders_metrics):
    df = pd.read_excel(wwm.path_full_YA_df,
                       sheet_name='ya_orders_metrics',
                       dtype={'Артикул': 'str'})
    df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
    df = wwm.clear_article(df, 'Артикул')
    if len(wwm.date_res_table(df)) != len(df.columns[1:]):
        exit()
    wwm.unload_metrics(path_metrics=path_ya_orders_metrics, 
                       data1=df, 
                       column='Артикул')
    T_ya_orders_metrics = wwm.get_T_metrics(path_metrics=path_ya_orders_metrics)
    sh = gc.open("full_YA")
    list_name = 'ya_orders_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_ya_orders_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)
    
def ya_fbs_stocks_metrics(path_ya_fbs_stocks_metrics):
    df = pd.read_excel(wwm.path_full_YA_df,
                       sheet_name='stocks_fbs_ya_metrics',
                       dtype={'Артикул': 'str'})
    df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
    df = wwm.clear_article(df, 'Артикул')
    df = df.fillna(0)
    df = df.groupby('Артикул').sum().reset_index(drop=False)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
    df = wwm.recovery_stocks_days(df)
    wwm.unload_metrics(path_metrics=path_ya_fbs_stocks_metrics, 
                       data1=df, 
                       column='Артикул')
    if len(wwm.date_res_table(df)) != len(df.columns[1:]):
        df = wwm.recovery_stocks_days(df)
        wwm.unload_metrics(path_metrics=path_ya_fbs_stocks_metrics, 
                           data1=df, 
                           column='Артикул')
    else:
        None
    T_ya_fbs_stocks_metrics = wwm.get_T_metrics(path_metrics=path_ya_fbs_stocks_metrics)
    sh = gc.open("full_YA")
    list_name = 'ya_fbs_stocks_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_ya_fbs_stocks_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)
    
def ya_fbo_stocks_metrics(path_ya_fbo_stocks_metrics):    
    df = pd.read_excel(wwm.path_full_YA_df,
                       sheet_name='stocks_ya_metrics',
                       dtype={'Артикул': 'str'})
    df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
    df = wwm.clear_article(df, 'Артикул')
    df = df.fillna(0)
    df = df.groupby('Артикул').sum().reset_index(drop=False)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
    df = wwm.recovery_stocks_days(df)
    if len(wwm.date_res_table(df)) != len(df.columns[1:]):
        exit()
    wwm.unload_metrics(path_metrics=path_ya_fbo_stocks_metrics, 
                       data1=df, 
                       column='Артикул')
    T_ya_fbo_stocks_metrics = wwm.get_T_metrics(path_metrics=path_ya_fbo_stocks_metrics)
    sh = gc.open("full_YA")
    list_name = 'ya_fbo_stocks_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_ya_fbo_stocks_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)
    
def ya_price_metrics(path_ya_price_metrics):
    df = pd.read_excel(wwm.path_full_YA_df,
                       sheet_name='ya_price_metrics',
                       dtype={'Артикул': 'str'})
    df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])].fillna(0)
    df = wwm.clear_article(df, 'Артикул').groupby('Артикул').mean().reset_index(drop=False)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
    df = wwm.recovery_price_days(df) 
    if len(wwm.date_res_table(df)) != len(df.columns[1:]):
        exit()
    wwm.unload_metrics(path_metrics=path_ya_price_metrics, 
                       data1=df, 
                       column='Артикул')
    T_ya_price_metrics = wwm.get_T_metrics(path_metrics=path_ya_price_metrics)
    sh = gc.open("full_YA")
    list_name = 'ya_price_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_ya_price_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)
    
# with DAG(
#     dag_id='full_YA_dag',
#     default_args=args,
#     schedule_interval='50 05 * * *',
#     start_date=datetime.datetime(2023, 11, 13),
#     catchup=False,
#     dagrun_timeout=datetime.timedelta(minutes=10)
# ) as dag:

#     ya_orders_metrics_task = PythonOperator(task_id = 'ya_orders_metrics',
#                                             python_callable=ya_orders_metrics, 
#                                             op_kwargs={'path_ya_orders_metrics': wwm.path_ya_orders_metrics},
#                                             dag=dag)
    
#     ya_fbs_stocks_metrics_task = PythonOperator(task_id = 'ya_fbs_stocks_metrics',
#                                           python_callable=ya_fbs_stocks_metrics, 
#                                           op_kwargs={'path_ya_fbs_stocks_metrics': wwm.path_ya_fbs_stocks_metrics},
#                                           dag=dag)
    
#     ya_fbo_stocks_metrics_task = PythonOperator(task_id = 'ya_fbo_stocks_metrics',
#                                           python_callable=ya_fbo_stocks_metrics, 
#                                           op_kwargs={'path_ya_fbo_stocks_metrics': wwm.path_ya_fbo_stocks_metrics},
#                                           dag=dag)
    
#     ya_price_metrics_task = PythonOperator(task_id = 'ya_price_metrics',
#                                            python_callable=ya_price_metrics, 
#                                            op_kwargs={'path_ya_price_metrics': wwm.path_ya_price_metrics},
#                                            dag=dag)
    
#     ya_orders_metrics_task >> \
#     ya_fbs_stocks_metrics_task >> \
#     ya_fbo_stocks_metrics_task >> \
#     ya_price_metrics_task

In [6]:
df = pd.read_excel(wwm.path_full_YA_df,
                   sheet_name='ya_orders_metrics',
                   dtype={'Артикул': 'str'})
df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
df = wwm.clear_article(df, 'Артикул')
# if len(wwm.date_res_table(df)) != len(df.columns[1:]):
#     exit()

In [7]:
wwm.unique_items(df)

In [8]:
df

,Артикул,13-March-2024,14-March-2024,15-March-2024,16-March-2024,17-March-2024,18-March-2024,19-March-2024,20-March-2024,21-March-2024,24-March-2024
0,16801,0,0,0,0,0,0,0,0,0,0
1,16802,0,0,0,0,0,0,0,0,0,0
2,16803,0,0,0,0,0,0,0,0,0,0
3,16804,0,0,0,0,0,0,0,0,0,0
4,18301,1,1,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1147,G0951,0,0,0,0,0,0,0,0,0,0
1148,Z0402,0,0,0,0,0,0,0,0,0,0
1149,G0638,0,1,1,0,1,0,1,1,0,0
1150,50976,0,0,0,0,0,0,0,0,0,0


In [2]:
ya_orders_metrics(wwm.path_ya_orders_metrics)

Были изменения, датафрейм выгружен


In [3]:
ya_fbs_stocks_metrics(wwm.path_ya_fbs_stocks_metrics)

100%|██████████████████████████████████████| 1155/1155 [00:01<00:00, 738.56it/s]


Были изменения, датафрейм выгружен


In [4]:
ya_fbo_stocks_metrics(wwm.path_ya_fbo_stocks_metrics)

100%|██████████████████████████████████████| 1155/1155 [00:01<00:00, 743.08it/s]


Были изменения, датафрейм выгружен


In [11]:
ya_price_metrics(wwm.path_ya_price_metrics)

100%|███████████████████████████████████████████| 14/14 [00:01<00:00,  8.10it/s]


Были изменения, датафрейм выгружен


In [6]:
df = pd.read_excel(wwm.path_full_YA_df,
                   sheet_name='ya_price_metrics',
                   dtype={'Артикул': 'str'})
df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])].fillna(0)
df = wwm.clear_article(df, 'Артикул').groupby('Артикул').mean().reset_index(drop=False)
df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)

In [9]:
df

,Артикул,21-March-2024,22-March-2024,23-March-2024,24-March-2024,25-March-2024,26-March-2024,27-March-2024,28-March-2024,29-March-2024,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024
0,16801,649,649,649,649,649,649,649,649,649,649,649,649,649,649
1,16802,649,649,649,649,649,649,649,649,649,649,649,649,649,649
2,16803,722,722,722,722,722,722,722,722,722,722,722,722,722,722
3,16804,601,601,601,601,601,601,601,601,601,601,601,601,601,601
4,16920,184,184,184,184,184,184,184,184,184,184,184,184,184,184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,Z0400,738,738,738,738,738,738,738,738,738,738,738,738,738,738
1151,Z0401,1380,1380,1380,1380,1380,1380,1380,1380,1380,1380,1380,1380,1380,1380
1152,Z0402,3900,3900,3900,3900,3900,3900,3900,3900,3900,3900,3900,3900,3900,3900
1153,Z0500,3510,3510,3510,3510,3510,3510,3510,3510,3510,3510,3510,3510,3510,3510


In [ ]:
    wwm.unload_metrics(path_metrics=path_ya_price_metrics, 
                       data1=df, 
                       column='Артикул')
    T_ya_price_metrics = wwm.get_T_metrics(path_metrics=path_ya_price_metrics)
    sh = gc.open("full_YA")
    list_name = 'ya_price_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_ya_price_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)

In [8]:
df = wwm.recovery_price_days(df) 

100%|███████████████████████████████████████████| 14/14 [00:01<00:00,  7.76it/s]


In [9]:
df = pd.read_excel(wwm.path_full_YA_df,
                   sheet_name='stocks_ya_metrics',
                   dtype={'Артикул': 'str'})
df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
df = wwm.clear_article(df, 'Артикул')
df = df.fillna(0)
df = df.groupby('Артикул').sum().reset_index(drop=False)
df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
# if len(wwm.date_res_table(df)) != len(df.columns[1:]):
#     exit()

In [11]:
df = wwm.recovery_stocks_days(df)

100%|█████████████████████████████████████| 1152/1152 [00:00<00:00, 1969.64it/s]


,Артикул,06-March-2024,07-March-2024,08-March-2024,09-March-2024,10-March-2024,11-March-2024,12-March-2024,13-March-2024,14-March-2024,15-March-2024,16-March-2024,17-March-2024
0,16801,0,0,0,0,0,0,0,0,0,0,0,0
1,16802,0,0,0,0,0,0,0,0,0,0,0,0
2,16803,0,0,0,0,0,0,0,0,0,0,0,0
3,16804,0,0,0,0,0,0,0,0,0,0,0,0
4,16920,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,Z0400,0,32,32,32,32,30,28,28,26,0,0,23
1148,Z0401,101,88,88,88,88,89,88,87,87,0,0,82
1149,Z0402,0,0,0,0,0,0,0,0,0,0,0,0
1150,Z0500,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
df

,Артикул,06-March-2024,07-March-2024,08-March-2024,09-March-2024,10-March-2024,11-March-2024,12-March-2024,13-March-2024,14-March-2024,17-March-2024
0,16801,0,0,0,0,0,0,0,0,0,0
1,16802,0,0,0,0,0,0,0,0,0,0
2,16803,0,0,0,0,0,0,0,0,0,0
3,16804,0,0,0,0,0,0,0,0,0,0
4,16920,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1147,Z0400,0,32,32,32,32,30,28,28,26,23
1148,Z0401,101,88,88,88,88,89,88,87,87,82
1149,Z0402,0,0,0,0,0,0,0,0,0,0
1150,Z0500,0,0,0,0,0,0,0,0,0,0
